In [1]:
import MyPackage
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

In [2]:
__mypath__ = MyPackage.MyClass_Path.MyClass_Path()  #路径类
myfile = MyPackage.MyClass_File.MyClass_File()  #文件操作类
myplt = MyPackage.MyClass_Plot.MyClass_Plot()  #直接绘图类(单个图窗)
myfig = MyPackage.MyClass_Plot.MyClass_Figure(AddFigure=False)  #对象式绘图类(可多个图窗)
mypltpro = MyPackage.MyClass_PlotPro.MyClass_PlotPro()  #Plot高级图系列
myfigpro = MyPackage.MyClass_PlotPro.MyClass_FigurePro(AddFigure=False)  #Figure高级图系列
mynp = MyPackage.MyClass_Array.MyClass_NumPy()  #多维数组类(整合Numpy)
mypd = MyPackage.MyClass_Array.MyClass_Pandas()  #矩阵数组类(整合Pandas)
mypdpro = MyPackage.MyClass_ArrayPro.MyClass_PandasPro()  #高级矩阵数组类
mytime = MyPackage.MyClass_Time.MyClass_Time()  #时间类
myDA = MyPackage.MyClass_DataAnalysis.MyClass_DataAnalysis()  #数据分析类
myBTE = MyPackage.MyClass_BackTestEvent.MyClass_BackTestEvent()  # 事件驱动型回测类
myBTV = MyPackage.MyClass_BackTestVector.MyClass_BackTestVector() # 向量型回测类
myWebQD = MyPackage.MyClass_WebCrawler.MyClass_WebQuotesDownload()  #金融行情下载类

In [3]:
Path = "C:\\Users\\i2011\\OneDrive\\Book_Code&Data\\量化投资以python为工具\\数据及源代码\\033"
CJSecurities = pd.read_csv(Path + '\\CJSecurities.csv', index_col=1, parse_dates=True)
CJSecurities = CJSecurities.iloc[:, 1:]
data0 = CJSecurities

In [5]:
data0

,Open,High,Low,Close,Volume
Date,,,,,
2014-01-02,10.42,10.42,10.20,10.27,54734700
2014-01-03,10.21,10.22,9.86,9.88,88544000
2014-01-06,9.88,9.97,9.71,9.80,67493100
2014-01-07,9.69,9.83,9.60,9.61,52957200
2014-01-08,9.67,9.91,9.65,9.77,55127700
2014-01-09,9.73,9.86,9.44,9.50,55607300
2014-01-10,9.50,9.55,9.24,9.28,44695000
2014-01-13,9.27,9.36,9.19,9.34,37213400
2014-01-14,9.30,9.49,9.27,9.43,39850200


In [6]:
class ABCStrategy(myBTE.bt.Strategy):
    # ---设定参数，必须写params，以self.params.Para0索引，可用于优化，内部必须要有逗号
    params = (('Para0', 15),)

    # ---只开头执行一次
    def __init__(self):
        print("init", len(self))
        self.smahandle = myBTE.addIndi_SMA(0, period=self.params.Para0)
        self.sma = lambda x: self.smahandle[-x]
        self.barscount = 0
        self.close = self.datas[0].close

    # ---每一个Bar迭代执行一次。next()执行完就进入下一个bar
    def next(self):
        print("next sma:", self.sma(0), self.sma(1), self.sma(2))
        if not self.position:
            if self.close[0] > self.sma(0):
                self.buy()
        else:
            if len(self) >= self.barscount + 5:
                self.sell()

    # ---策略每笔订单通知函数。已经进入下一个bar，且在next()之前执行
    def notify_order(self, order):
        if myBTE.order_status_check(order, True) == True:
            self.barscount = len(self)

    # ---策略每笔交易通知函数。已经进入下一个bar，且在notify_order()之后，next()之前执行。
    def notify_trade(self, trade):
        myBTE.trade_status(trade, isclosed=False)

    # ---策略加载完会触发此语句
    def stop(self):
        print("stop(): ", myBTE.ValueCash(), self.sma(0), self.sma(1), self.sma(2))

In [8]:
if __name__ == '__main__':
    # ---基础设置
    myBTE = MyPackage.MyClass_BackTestEvent.MyClass_BackTestEvent()  # 回测类
    myBTE.ValueCash(100000)
    myBTE.setcommission(0.001)
    myBTE.AddBarsData(data0, fromdate=None, todate=None)

    myBTE.addstrategy(ABCStrategy)
    myBTE.run(plot=True,)

    #myBTE.optstrategy(ABCStrategy,Para0=range(5,100))
    #myBTE.run(maxcpus=None,plot = False)

init 0
next sma: 9.556000000000001 nan nan
Buy/Sell order submitted to broker / accepted by broker - waiting...
Buy/Sell order submitted to broker / accepted by broker - waiting...
BUY 执行成功，Price: 9.66, Cost: 9.66, Commission 0.01
每单交易利润, GROSS = 0.00, NET = -0.01
next sma: 9.510666666666667 9.556000000000001 nan
next sma: 9.503333333333334 9.510666666666667 9.556000000000001
next sma: 9.482666666666667 9.503333333333334 9.510666666666667
next sma: 9.475999999999999 9.482666666666667 9.503333333333334
next sma: 9.463333333333333 9.475999999999999 9.482666666666667
next sma: 9.451333333333334 9.463333333333333 9.475999999999999
Buy/Sell order submitted to broker / accepted by broker - waiting...
Buy/Sell order submitted to broker / accepted by broker - waiting...
SELL 执行成功，Price: 9.32, Cost: 9.66, Commission 0.01
每单交易利润, GROSS = -0.34, NET = -0.36
next sma: 9.454 9.451333333333334 9.463333333333333
next sma: 9.452666666666666 9.454 9.451333333333334
next sma: 9.445333333333334 9.4526666